Ref: https://github.com/openai/openai-cookbook/blob/main/examples/gpt4o/introduction_to_gpt4o.ipynb

In [1]:
from openai import OpenAI 
import os
import os
from openai import AzureOpenAI
from azure.identity import DefaultAzureCredential, get_bearer_token_provider
import time
import cv2
from moviepy.editor import VideoFileClip
import time
import base64
import gradio as gr
import base64
import os
import cv2
from moviepy.editor import VideoFileClip
from PIL import Image
from io import BytesIO
import pickle
from dotenv import load_dotenv
import json
import  requests
import requests
import json
import urllib.request
import re


In [2]:
output_dir = "./processed_files"
os.makedirs(output_dir, exist_ok=True)

In [3]:
load_dotenv(override=True)

endpoint = os.getenv("OPEN_AI_ENDPOINT")
api_key = os.getenv("OPEN_AI_KEY")
model_ep = os.getenv("MODEL_EP")
model_key = os.getenv("MODEL_KEY")

deployment_azure = os.getenv("OPEN_AI_DEPLOYMENT")
      
token_provider = get_bearer_token_provider(DefaultAzureCredential(), "https://cognitiveservices.azure.com/.default")
      
client_azure = AzureOpenAI(
    azure_endpoint=endpoint,
    api_key=api_key,
    api_version="2024-02-01",
)



In [4]:
#!ffmpeg -i data/FY25Strategy_compressed.mp4 -vcodec libx264 -preset ultrafast -crf 28 -acodec aac -b:a 128k FY25Strategy_compressed1.mp4



In [5]:
def save_pickle(data, file_path):
    with open(file_path, 'wb') as file:
        pickle.dump(data, file)

def load_pickle(file_path):
    with open(file_path, 'rb') as file:
        return pickle.load(file)

def get_pickle_path(video_name, start, end):
    """Generates a file path for the pickle file based on the frame range."""
    return os.path.join(output_dir, f"{video_name}_summary_{start}_{end}.pkl")

def get_final_summary_path(video_name):
    """Generates a file path for the pickle file based on the frame range."""
    return os.path.join(output_dir, f"{video_name}_final_summary.pkl")

def get_phi3_final_summary_path(video_name):
    """Generates a file path for the pickle file based on the frame range."""
    return os.path.join(output_dir, f"{video_name}_phi3_final_summary.pkl")

def get_phi3_pickle_path(video_name, start, end):
    """Generates a file path for the pickle file based on the frame range."""
    return os.path.join(output_dir, f"{video_name}_phi3_summary_{start}_{end}.pkl")



In [6]:



def get_response(input_data, ep=model_ep, key=model_key):
    #print(input_data)
    body = str.encode(json.dumps(input_data))
    if not model_key:
        raise Exception("A key should be provided to invoke the endpoint")
    headers = {'Content-Type':'application/json', 'Authorization':('Bearer '+ key)}
    req = urllib.request.Request(ep, body, headers)

    try:
        response = urllib.request.urlopen(req)
        result = response.read()
        return json.loads(result.decode("utf8", 'ignore'))
    except urllib.error.HTTPError as error:
        print("The request failed with status code: " + str(error.code))
        print(error.info())
        print(error.read().decode("utf8", 'ignore'))
        
def get_frameset_summary_phi3(pickle_file_path, base64Frames, start, end, regen=False, in_prompt=""):
    print(pickle_file_path)
    if regen==False and os.path.exists(pickle_file_path):
        content = load_pickle(pickle_file_path)
        print(f"Loaded existing summary for frames {start} to {end}.")
    else:
        if in_prompt == "":
            in_prompt = """Extract following details about the car in the image:
1. Whether the car is facing sideways, from or back. 
2. Dents /Scrtches visible in the car detailing where thet are visible.
3. Shattered windows of the car, including details of which windows of the car.
4. Damages to front/back bumber of the car.
5. Do not make up any info, stay grounded to the provided image.
6. Number plate of the car.
7. Any other info that would be of interest to insurance company.
6. Stay truthful."""
        user_content=[
            {
              "type": "text",
              "text": in_prompt
            },
            *map(lambda x: {"type": "image_url", 
                            "image_url": {"url": f'data:image/jpg;base64,{x}'}}, base64Frames[start:end])
        ]
       
        
        input_data = {
            "input_data": {
            "input_string": [
              {
                "role": "user",
                "content": user_content
              }
            ],
            "parameters": { "temperature": 0.7, "max_new_tokens": 2048 }
            }
        }
        
        
        results = get_response(input_data)
        print(results)
        
        content = results["output"]
        save_pickle(content, pickle_file_path) 
    return content

In [7]:
if 1==2:
    base64Frames = []
    pickle_file_path = "test.pkl"
    file_path = "processed_files/car_frame_75.jpg"
    with open(file_path, "rb") as file:
        buffer = file.read()
        base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
    in_prompt = """Extract following details about the car in the image:
    1. Whether the car is facing sideways, from or back. 
    2. Dents /Scrtches visible in the car detailing where thet are visible.
    3. Shattered windows of the car, including details of which windows of the car.
    4. Damages to front/back bumber of the car.
    5. Do not make up any info, stay grounded to the provided image.
    6. Number plate of the car.
    7. Any other info that would be of interest to insurance company.
    6. Stay truthful."""
    get_frameset_summary_phi3(pickle_file_path, base64Frames, 0, 1, True, in_prompt)

In [8]:


# Ensure the short path for saving processed files


all_responses= []
all_phi3_responses= []
def get_frameset_summary(pickle_file_path, base64Frames, start, end):
    print(pickle_file_path)
    if os.path.exists(pickle_file_path):
        content = load_pickle(pickle_file_path)
        #response = eval(response)
        print(f"Loaded existing summary for frames {start} to {end}.")
    else:
        try:
            messages=[
                {"role": "system", "content": """You generate detailed note for inspection of a car for insurance company. 
    You keep the note grounded on info provided to you as Inspection details. 
    You respond in Markdown/bulleted list of points so as to make the inpformation easy to understand.
    """},
                {"role": "user", "content": [
                    {
                      "type": "text",
                      "text": "These are the frames from the video."
                    },
                    *map(lambda x: {"type": "image_url", 
                                    "image_url": {"url": f'data:image/jpg;base64,{x}', "detail": "high"}}, base64Frames[start:end])
                    ],
                }
                ]

            response = client_azure.chat.completions.create(
                model=deployment_azure,
                messages=messages,
                temperature=0,
            )
            #print(response.choices[0].message.content)
            save_pickle(response.choices[0].message.content, pickle_file_path) 
            content = response.choices[0].message.content
        except:
            content=""
    return content

def get_final_summary(base_video_path, in_responses, pickle_file_path):
    
    if os.path.exists(pickle_file_path):
        content = load_pickle(pickle_file_path)
        return content
    all_notes = "\nNotes:\n".join(in_responses)
    
    try:
        messages=[
            {"role": "system", "content": """
You generate detailed note for inspection of a car for insurance company. 
You keep the note grounded on info provided to you as Inspection details. 
You create a consolidated summary based on notes created by inspecting car from different angles.
All notes are for same car.
You respond in html tabluar format so as to make the information easy to understand.
"""},
            {"role": "user", "content": [
                {
                  "type": "text",
                  "text": f"Inspection details: {all_notes}"
                },
                ],
            }
            ]

        response = client_azure.chat.completions.create(
            model=deployment_azure,
            messages=messages,
            temperature=0,
        )
        #print(response.choices[0].message.content)
        content = response.choices[0].message.content
        save_pickle(content, pickle_file_path) 
    except:
        content=""
    return content
        
        
def summarize(in_gallery):
    base64Frames = []
    #print(in_gallery)
    base_video_path = ""
    # Convert file paths to base64-encoded frames
    for file_path in in_gallery:
        print(file_path)
        # If item is a tuple, unpack it
        #if isinstance(file_path, dict) and 'name' in file_path:
        #    file_path = file_path['name']
        #    print("dict")
        #elif isinstance(file_path, tuple):
        file_path = file_path[0]
            
        if base_video_path == "":
            #print(file_path)
            base_video_path = file_path.split(".")[0].split("\\")[-1]
            #print(f"Base File Path: {base_video_path}")
        with open(file_path, "rb") as file:
            buffer = file.read()
            base64Frames.append(base64.b64encode(buffer).decode("utf-8"))
            
    
    start = 0
    batch_size = 2
    end = start + batch_size
    print(f"No. of frames:{len(base64Frames)}")
    while end<= len(base64Frames):
        
        pickle_file_path = get_pickle_path(base_video_path, start, end)
        content = get_frameset_summary(pickle_file_path, base64Frames, start, end)
        
        phi3_pickle_file_path = get_phi3_pickle_path(base_video_path, start, end)
        phi3_content = get_frameset_summary_phi3(phi3_pickle_file_path, base64Frames, start, end)
        
        
        all_responses.append(content)
        all_phi3_responses.append(phi3_content)
        if end == len(base64Frames):
            break;
        end += batch_size
        if end > len(base64Frames):
            end = len(base64Frames)
        start += batch_size
        
    pickle_file_path = get_final_summary_path(base_video_path)
    phi3_pickle_file_path = get_phi3_final_summary_path(base_video_path)
    print(all_phi3_responses)
    final_summary = get_final_summary(base_video_path, all_responses, pickle_file_path)
    final_summary_phi3 = get_final_summary(base_video_path, all_phi3_responses, phi3_pickle_file_path)
    return "Final Summary:" + final_summary, "Final Summary:" + final_summary_phi3



def process_video(video_path, seconds_per_frame=2):
    frame_paths = []
    video_name = os.path.basename(video_path)
    base_video_path = os.path.join(output_dir, os.path.splitext(video_name)[0])


    video = cv2.VideoCapture(video_path)
    total_frames = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    fps = video.get(cv2.CAP_PROP_FPS)
    frames_to_skip = int(fps * seconds_per_frame)
    curr_frame = 0

    # Loop through the video and extract frames at specified sampling rate
    while curr_frame < total_frames - 1:
        frame_path = f"{base_video_path}_frame_{curr_frame}.jpg"
        #print(f"Frame  {frame_path}.")
        if os.path.exists(frame_path):
            print(print(f"Frame  {frame_path} already exists."))
        else:
            video.set(cv2.CAP_PROP_POS_FRAMES, curr_frame)
            success, frame = video.read()
            if not success:
                break
            cv2.imwrite(frame_path, frame)
        frame_paths.append(frame_path)
        curr_frame += frames_to_skip
    video.release()

    # Extract audio from video
    audio_path = f"{base_video_path}.mp3"
    if os.path.exists(audio_path):
        print(f"Audio file {audio_path} already exists.")
    else:
        clip = VideoFileClip(video_path)
        clip.audio.write_audiofile(audio_path, bitrate="32k")
        clip.audio.close()
        clip.close()

    print(f"Extracted {len(frame_paths)} frames")
    print(f"Extracted audio to {audio_path}")
    return frame_paths, audio_path

def handle_video_upload(video_file_path):
    frame_paths, audio_path = process_video(video_file_path, seconds_per_frame=0.5)
    return frame_paths, audio_path


def upload_video(video):
    print(video)
    final_video = video
    if video:
        final_video = video.replace("\\", "/")
        print(final_video)
    return gr.update(True),gr.update(False), final_video, video

custom_css = """
#input-video video {
    height: 400px;  /* Set your desired max height here */
}
#output-video video {
    height: 400px;  /* Set your desired max height here */
}
"""
# Create the Gradio Blocks interface
with gr.Blocks() as demo:
    gr.Markdown("## Unleash Innovation with Azure Open AI")
    with gr.Row():
        with gr.Column():
            video_input = gr.Video()
            #video_output = gr.Video()
    
    
    process_button = gr.Button("Process")
    
    with gr.Row():
        with gr.Column():
            output_gallery = gr.Gallery(label="Extracted Frames")
    with gr.Row():
        with gr.Column():
            txt_summary = gr.HTML(label="Video Summary")
        with gr.Column():
            txt_summary_phi3 = gr.HTML(label="Video Summary - Phi3")
    with gr.Row():
        output_audio = gr.Audio(label="Extracted Audio")
    with gr.Row():
        summarize_button = gr.Button("Summarize")

    #video_input.change(fn=upload_video, inputs=video_input, outputs=[video_input,video_output,video_input, video_output])
    #video_input.change(fn=upload_video, inputs=video_input, outputs=[video_input,video_input,video_input, video_input])
    process_button.click(fn=handle_video_upload, inputs=video_input, outputs=[output_gallery, output_audio])
    summarize_button.click(fn=summarize, inputs=output_gallery, outputs=[txt_summary,txt_summary_phi3])

# Launch the Gradio app
demo.launch(share=True)


Running on local URL:  http://127.0.0.1:7862

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.


Traceback (most recent call last):
  File "C:\Users\shagrawal\Miniconda3\envs\va\Lib\urllib\request.py", line 1348, in do_open
    h.request(req.get_method(), req.selector, req.data, headers,
  File "C:\Users\shagrawal\Miniconda3\envs\va\Lib\http\client.py", line 1303, in request
    self._send_request(method, url, body, headers, encode_chunked)
  File "C:\Users\shagrawal\Miniconda3\envs\va\Lib\http\client.py", line 1349, in _send_request
    self.endheaders(body, encode_chunked=encode_chunked)
  File "C:\Users\shagrawal\Miniconda3\envs\va\Lib\http\client.py", line 1298, in endheaders
    self._send_output(message_body, encode_chunked=encode_chunked)
  File "C:\Users\shagrawal\Miniconda3\envs\va\Lib\http\client.py", line 1058, in _send_output
    self.send(msg)
  File "C:\Users\shagrawal\Miniconda3\envs\va\Lib\http\client.py", line 996, in send
    self.connect()
  File "C:\Users\shagrawal\Miniconda3\envs\va\Lib\http\client.py", line 1468, in connect
    super().connect()
  File "C:\Us

# Whats New 
https://techcommunity.microsoft.com/t5/ai-azure-ai-services-blog/announcing-multimodal-innovations-in-generative-ai-with-azure/ba-p/4146804
1. GPT4o will have global deployments - Public Preview 
2. Customers will be able to purchase today - 21 May 24
3. Multi Modality 
4. Option to turn it off to be available in few weeks
5. Image Recognition 